Importing library

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

Importing dataset

In [2]:
dataset = pd.read_csv('/kaggle/input/sentiment-dataset/train.csv');
dataset_test = pd.read_csv('/kaggle/input/sentiment-dataset/test.csv');

In [3]:
dataset.isnull().sum()

Unnamed: 0    0
Text          0
label         0
dtype: int64

In [4]:
dataset_test.head(3)

,id,Text
0,0,I'm worried about the impact of panic on my re...
1,1,Trying to maintain my equilibrium when emotion...
2,2,Feeling like I'm in a constant struggle to fin...


In [5]:
dataset['label'].unique()

array(['Depression', 'Anxiety Disorder', 'Panic Disorder',
       'Anger/ Intermittent Explosive Disorder', 'Narcissistic Disorder'],
      dtype=object)

In [6]:
len(dataset)

785

Cleaning the texts

In [7]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []  # This list will contain our cleaned texts/reviews

# Assuming dataset is a DataFrame and 'column_name' is the column containing text data
for i in range(0, len(dataset)):
    try:
        review = str(dataset.iloc[i, 1:-1])  # Convert to string
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        review = ""

    review = re.sub('[^a-zA-Z]', ' ', review)  # Replacing all the punctuations with spaces (' ')
    review = review.lower()  # Converting all the letters into smaller case
    review = review.split()  # Splitting the string of ith review
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')

    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


Creating bag of words model(spars matrix)

In [8]:
len(corpus)

785

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 826);
x = cv.fit_transform(corpus).toarray();
y = dataset.iloc[:, -1].values;

In [10]:
print(len(x[0]))

826


OneHot encoding Categorical data

In [11]:
# from sklearn.preprocessing import OneHotEncoder

# # Initialize the OneHotEncoder
# encoder = OneHotEncoder()

# # Fit and transform the encoder on your single-column array
# y = encoder.fit_transform(y.reshape(-1, 1)).toarray()

In [12]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Fit the encoder to your y data and transform it
y = label_encoder.fit_transform(y.reshape(-1, 1))

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
print(y)

[2 1 2 4 4 4 4 2 1 0 2 0 2 1 0 3 0 0 1 3 0 0 2 1 4 1 0 0 4 3 2 2 0 2 3 3 1
 1 3 2 2 3 4 1 2 3 2 3 1 0 2 3 3 2 1 3 2 4 3 4 4 4 2 2 0 2 2 2 2 0 4 2 1 2
 3 2 1 4 1 3 2 1 0 0 4 2 1 3 1 2 3 4 3 1 2 1 4 0 4 1 3 4 2 1 1 3 4 4 0 2 2
 0 1 4 3 2 1 4 0 3 1 3 2 2 3 4 2 1 4 0 2 1 3 1 0 1 3 0 4 2 2 0 2 2 0 4 0 0
 3 2 3 2 2 0 3 1 0 4 2 1 1 2 3 0 2 3 3 0 2 0 3 0 1 4 0 2 1 2 4 0 3 2 0 0 2
 0 3 4 0 2 4 3 1 3 1 4 0 4 2 3 2 2 3 2 4 3 2 2 1 1 3 0 1 1 3 2 2 2 2 1 1 4
 3 1 2 2 2 1 2 0 0 2 1 4 3 1 0 0 0 2 0 4 4 2 3 0 0 0 2 2 4 2 0 3 2 0 2 3 2
 0 2 3 2 0 4 2 2 3 4 3 0 1 1 2 1 1 0 2 2 1 3 0 0 1 2 3 3 2 4 4 4 0 0 3 3 1
 1 2 0 2 3 4 3 3 2 2 2 1 0 2 2 2 0 2 2 3 0 2 2 0 3 4 2 0 1 1 3 0 1 2 0 2 3
 1 4 4 4 0 0 3 1 4 2 3 0 3 0 4 2 2 2 4 2 4 3 0 0 3 2 0 2 4 0 0 1 2 2 2 1 3
 0 1 2 0 1 2 2 2 3 4 0 0 3 2 4 3 0 3 2 2 0 1 1 0 0 0 3 3 1 1 3 3 0 2 0 1 2
 3 0 1 1 0 0 3 4 1 3 0 3 1 4 3 2 3 1 3 1 4 3 3 3 1 4 2 2 0 2 3 4 4 4 2 3 1
 2 1 4 4 1 3 1 1 1 3 1 1 2 2 4 0 2 3 4 3 0 4 3 0 1 2 4 2 1 0 2 3 3 3 4 1 2
 3 2 3 0 2 4 1 2 4 3 3 2 

In [14]:
len(x), len(y)

(785, 785)

Splitting the data into training and test set

In [15]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state=0);

Feature scaling for better performance of the ANN

In [16]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler();
# x_train = sc.fit_transform(x_train);
# x_test = sc.fit_transform(x_test);

In [17]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler();
x = sc.fit_transform(x);

In [20]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter = 1000)
classifier.fit(x, y)

LogisticRegression(max_iter=1000)

Predicting New Result

In [32]:
new_review = "I'm constantly anticipating the next panic attack."
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(sc.transform(new_X_test))

class_labels = ["Anger/ Intermittent Explosive Disorder", "Anxiety Disorder", "Depression", "Narcissistic Disorder", "Panic Disorder"]

# predicted_index = np.argmax(new_y_pred)

# Get the corresponding class label
predicted_label = class_labels[new_y_pred[0]]

# Print the predicted label
print(predicted_label)

Panic Disorder


index

In [ ]:
new_review = "I'm constantly anticipating the next panic attack."
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(sc.transform(new_X_test))
print(new_y_pred)

In [ ]:
# from sklearn import metrics
# y_pred = classifier.predict(x_test)
# print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

In [ ]:
testset = []
# Assuming dataset is a DataFrame and 'column_name' is the column containing text data
for i in range(0, len(dataset_test)):
    try:
        review = str(dataset_test.iloc[i, -1])  # Convert to string
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        review = ""

    review = re.sub('[^a-zA-Z]', ' ', review)  # Replacing all the punctuations with spaces (' ')
    review = review.lower()  # Converting all the letters into smaller case
    review = review.split()  # Splitting the string of ith review
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')

    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    testset.append(review)
    
res_X_test = cv.transform(testset).toarray()
res_y_pred = classifier.predict(sc.transform(res_X_test))

In [ ]:
type(res_y_pred)

In [ ]:
print(res_y_pred)

In [ ]:
# Define a dictionary to map encodings to labels
label_mapping = {
    0: "Anger/ Intermittent Explosive Disorder",
    1: "Anxiety Disorder",
    2: "Depression",
    3: "Narcissistic Disorder",
    4: "Panic Disorder"
}

# Replace the encodings in res_y_pred with labels
res_y_pred_labels = [label_mapping[encoding] for encoding in res_y_pred]

# res_y_pred_labels now contains the labels corresponding to the encodings
print(res_y_pred_labels)

In [ ]:
Submission = dataset_test[["id"]]
Submission["label"] = res_y_pred_labels
Submission

In [ ]:
Submission.to_csv("submission.csv", index = None)